#Paquetes necesarios

In [1]:
import cv2  
import math 

from ultralytics import YOLO

WARNING  Python>=3.10 is required, but Python==3.9.20 is currently installed 


Seguimiento. Requiere instalar lap con pip install lap

In [2]:
from collections import defaultdict
import numpy as np
import easyocr
# Variable booleana que indica si quieres guardar en memoria (False) o visualizar en pantalla (True)
display = False
#Carga del modelo de lengua
reader = easyocr.Reader(['es']) 

# Carga del modelo, descarga en disco si no está presente en la carpeta
model = YOLO('YOLO/yolo11n.pt') #Contenedores
license_plate_model = YOLO('YOLO/best.pt')

# Etiqueta de las distintas clases
classNames = ["person", "bicycle", "car", "motorbike", "bus", "truck"]


# Captura desde la webcam
vid = cv2.VideoCapture("YOLO/C0142.MP4")
if not display:
    frame_width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    output_video = cv2.VideoWriter('YOLO/resultados.avi', cv2.VideoWriter_fourcc(*'XVID'), 20, (frame_width, frame_height))
track_history = defaultdict(lambda: [])
  
while(True):      
    # fotograma a fotograma
    ret, img = vid.read()
  
    # si hay imagen válida
    if ret:  
        # Seguimiento, con persistencia entre fotogramas
        results = model.track(img, persist=True, classes = [0,1,2,3,5,7])
        
        # Para cada detección
        for r in results:
            boxes = r.boxes

            for box in boxes:
                # Contenedor
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # convert to int values

                #Etiqueta de seguimiento
                if box.id is not None:
                    track_id = str(int(box.id[0].tolist()))
                else:
                    track_id = ''
                
                # Confianza
                confidence = math.ceil((box.conf[0]*100))/100
                print("Confianza --->",confidence)

                # Clase
                cls = int(box.cls[0])
                print("Clase -->", classNames[cls])

                if cls >= 2:
                    # Recorta el contenedor del coche
                    car_crop = img[y1:y2, x1:x2]

                    # Detecta la matrícula en el contenedor del coche
                    license_plate_results = license_plate_model.predict(car_crop)
                    possible_licenses = {"Not Found": 0}
                    # Procesa cada detección en el contenedor del coche
                    for lp_result in license_plate_results:
                        lp_boxes = lp_result.boxes
                        for lp_box in lp_boxes:
                            lp_x1, lp_y1, lp_x2, lp_y2 = lp_box.xyxy[0]
                            lp_x1, lp_y1, lp_x2, lp_y2 = int(lp_x1), int(lp_y1), int(lp_x2), int(lp_y2)
                            #Reconocimiento del texto
                            res = reader.readtext(img[lp_y1:lp_y2, lp_x1:lp_x2])
                            for (bbox, text, prob) in res:
                                if possible_licenses[text] is not None and possible_licenses[text] < prob:
                                    possible_licenses[text] = prob
                            # Ajustar coordenadas al área de imagen completa
                            lp_x1 += x1
                            lp_x2 += x1
                            lp_y1 += y1
                            lp_y2 += y1

                            # Dibuja el contenedor de la matrícula sobre el coche
                            cv2.rectangle(img, (lp_x1, lp_y1), (lp_x2, lp_y2), (0, 255, 0), 2)
                            cv2.putText(img,
                                        f"Matricula: {max(possible_licenses, key=possible_licenses.get)}",
                                        (lp_x1, lp_y1 - 10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Convierte identificador numérico de clase a un color RGB
                escala = int((cls / len(classNames)) * 255 * 3)
                if escala >= 255*2:
                    R = 255
                    G = 255
                    B = escala - 255*2
                else:
                    if escala >= 255:
                        R = 255
                        G = escala - 255
                        B = 0
                    else:
                        R = escala
                        G = 0
                        B = 0

                # Dibuja el contenedor y clase
                cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 3)
                cv2.putText(img, track_id + ' ' + classNames[cls] , [x1, y1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, B), 2)

        # Muestra fotograma
        if display:
            cv2.imshow('Vid', img)
        else:
            # Muestra fotograma
            output_video.write(img)
        # Detenemos pulsado ESC
        if cv2.waitKey(20) == 27:
            break
    else:
        break
  
# Libera el objeto de captura
vid.release()
if display:
    cv2.destroyAllWindows()
else:
    output_video.release()  # Libera el archivo de video

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.



0: 384x640 1 car, 1 bus, 105.7ms
Speed: 8.0ms preprocess, 105.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.94
Clase --> truck

0: 352x416 1 License_Plate, 80.6ms
Speed: 0.0ms preprocess, 80.6ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 416)
Confianza ---> 0.85
Clase --> car

0: 320x416 (no detections), 97.3ms
Speed: 0.0ms preprocess, 97.3ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 416)

0: 384x640 1 car, 1 bus, 105.5ms
Speed: 0.0ms preprocess, 105.5ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)
Confianza ---> 0.94
Clase --> truck

0: 352x416 1 License_Plate, 65.0ms
Speed: 6.5ms preprocess, 65.0ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 416)
Confianza ---> 0.82
Clase --> car

0: 352x416 (no detections), 56.2ms
Speed: 0.0ms preprocess, 56.2ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 416)

0: 384x640 1 car, 1 bus, 73.0ms
Speed: 8.0ms preprocess, 73.0ms

KeyboardInterrupt: 